# Import libraries

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import make_scorer
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from skopt import BayesSearchCV

# Import data

In [2]:
data = pd.read_csv("covid_data_196.csv")

In [3]:
# Set variables
xvar = ["Employed", "Males", "Vulnerable_pop", "Health_Insurance", 
        "Secondary_Education", "Life_Expectancy", "prev_cronic_prov", 
        "Indice_Pobreza_Compuesto", "White_1000_Inhab", "Assian_1000_Inhab", 
        "Black_1000_Inhab", "prev_diferencial_prov_endes", "prev_hipertension_prov_endes", 
        "prev_diabetes_prov_endes", "prev_obesidad_prov_endes", "Days_Till_Attended", 
        "SD_Days_Till_Attended",  "Travel_Time_toHFacility_Hours", "SD_TTtHFH",
        "Waiting_Time_4Attention_Hours", "SD_WT4AH", "logPD_1000", "Overcrowding",
        "Natural_Region1", "Natural_Region2", "Natural_Region3"]

datac = data.dropna()

x = datac[xvar]
Y = datac["logmuertes1000"]

In [4]:
# Display dataset
datac

,_ID,_CX,_CY,IDDPTO,DEPARTAMEN,prov,PROVINCIA,CAPITAL,FUENTE,dep,...,casos_1000_Inhab,muertes_1000_Inhab,logcasos1000,logmuertes1000,dominio,Natural_Region,Natural_Region1,Natural_Region2,Natural_Region3,logPD_1000
0,1,-77.773167,-6.437059,1,AMAZONAS,101,CHACHAPOYAS,CHACHAPOYAS,INEI,1,...,37.995892,0.414370,3.637478,-0.880997,sierra norte,sierra,0,1,0,0.614774
1,2,-78.402692,-5.087450,1,AMAZONAS,102,BAGUA,BAGUA,INEI,1,...,108.002700,1.322537,4.682156,0.279552,selva,selva,0,0,1,1.538134
2,3,-77.873091,-5.683441,1,AMAZONAS,103,BONGARA,JUMBILLA,INEI,1,...,14.393260,0.390061,2.666760,-0.941452,selva,selva,0,0,1,2.541240
3,4,-78.038591,-4.166722,1,AMAZONAS,104,CONDORCANQUI,SANTA MARIA DE NIEVA,INEI,1,...,78.714386,0.518013,4.365826,-0.657755,selva,selva,0,0,1,-1.905125
4,5,-78.077829,-6.323335,1,AMAZONAS,105,LUYA,LAMUD,INEI,1,...,9.924386,0.247547,2.294995,-1.396155,sierra norte,sierra,0,1,0,0.998217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,191,-80.740769,-3.969757,24,TUMBES,2402,CONTRALMIRANTE VILLAR,ZORRITOS,INEI,24,...,22.177898,1.282234,3.099096,0.248604,costa norte,costa,1,0,0,0.231815
191,192,-80.256500,-3.652287,24,TUMBES,2403,ZARUMILLA,ZARUMILLA,INEI,24,...,31.876997,1.289821,3.461885,0.254503,costa norte,costa,1,0,0,0.147406
192,193,-74.058177,-8.672964,25,UCAYALI,2501,CORONEL PORTILLO,PUCALLPA,INEI,25,...,40.443245,0.739260,3.699900,-0.302106,selva,selva,0,0,1,0.893040
193,194,-73.218893,-10.389299,25,UCAYALI,2502,ATALAYA,ATALAYA,INEI,25,...,9.427460,0.141919,2.243627,-1.952501,selva,selva,0,0,1,0.625074


# Setting parameters

In [5]:
# Set the cross-validation

# Define cross-validation method
cv = RepeatedKFold(n_splits=4, n_repeats=5,random_state=0)

In [6]:
# Set the hyperparameters grid

# Define grid
param_grid = {
    'enr__alpha': (1e-2, 100.0, 'log-uniform'),
    'enr__l1_ratio': (0.1, 1.0, 'uniform')
}

# Model training

In [7]:
# Define evaluation and post-processing criteria
from sklearn.metrics import mean_squared_error

scoring = make_scorer(mean_squared_error, greater_is_better=False)

# Define the model
pipe = Pipeline([('scaler', StandardScaler()), ('enr', ElasticNet())])


# Define search
search_ddnn = BayesSearchCV(estimator = pipe, search_spaces=param_grid, scoring=scoring,
                           cv=cv, n_jobs=-1, verbose = 4, n_iter=50, n_points=5)

# Perform the search
results_ENLR = search_ddnn.fit(x, Y)

Fitting 20 folds for each of 5 candidates, totalling 100 fits
Fitting 20 folds for each of 5 candidates, totalling 100 fits
Fitting 20 folds for each of 5 candidates, totalling 100 fits
Fitting 20 folds for each of 5 candidates, totalling 100 fits
Fitting 20 folds for each of 5 candidates, totalling 100 fits
Fitting 20 folds for each of 5 candidates, totalling 100 fits
Fitting 20 folds for each of 5 candidates, totalling 100 fits
Fitting 20 folds for each of 5 candidates, totalling 100 fits
Fitting 20 folds for each of 5 candidates, totalling 100 fits
Fitting 20 folds for each of 5 candidates, totalling 100 fits


In [8]:
# Display search results
pd.DataFrame(results_ENLR.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_enr__alpha,param_enr__l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,...,split13_test_score,split14_test_score,split15_test_score,split16_test_score,split17_test_score,split18_test_score,split19_test_score,mean_test_score,std_test_score,rank_test_score
0,0.008703,0.008334,0.002171,0.000449,1.444971,0.901504,"{'enr__alpha': 1.4449707523957294, 'enr__l1_ra...",-0.875347,-0.599164,-0.848559,...,-0.795071,-0.778475,-0.644891,-0.823199,-0.517867,-1.014894,-0.796728,-0.785377,0.110622,43
1,0.005001,0.000895,0.001925,0.000745,0.22392,0.240954,"{'enr__alpha': 0.22392024838186864, 'enr__l1_r...",-0.220508,-0.277961,-0.276691,...,-0.231174,-0.274015,-0.235051,-0.256912,-0.138133,-0.417965,-0.266898,-0.267874,0.054221,27
2,0.004738,0.001066,0.001722,0.000474,6.631357,0.953533,"{'enr__alpha': 6.631356771134832, 'enr__l1_rat...",-0.875347,-0.599164,-0.848559,...,-0.795071,-0.778475,-0.644891,-0.823199,-0.517867,-1.014894,-0.796728,-0.785377,0.110622,43
3,0.005033,0.001261,0.001981,0.000678,0.113174,0.498979,"{'enr__alpha': 0.11317423788440324, 'enr__l1_r...",-0.206111,-0.275210,-0.267738,...,-0.220895,-0.269988,-0.227506,-0.255507,-0.144500,-0.398846,-0.263422,-0.261692,0.052331,20
4,0.004729,0.000964,0.001992,0.000550,0.023407,0.657269,"{'enr__alpha': 0.023406791210197442, 'enr__l1_...",-0.199398,-0.262892,-0.260744,...,-0.225338,-0.245631,-0.249255,-0.258612,-0.174057,-0.362279,-0.243139,-0.258791,0.044994,14
5,0.003851,0.000477,0.001600,0.000490,0.03382,0.7617,"{'enr__alpha': 0.03381960970969731, 'enr__l1_r...",-0.193261,-0.266297,-0.247591,...,-0.217787,-0.251041,-0.234398,-0.249512,-0.155964,-0.368188,-0.248811,-0.254168,0.047791,6
6,0.003898,0.000878,0.001300,0.000458,0.012948,0.543868,"{'enr__alpha': 0.01294820965003328, 'enr__l1_r...",-0.209881,-0.258438,-0.311776,...,-0.232682,-0.245575,-0.288863,-0.269144,-0.233369,-0.353645,-0.234752,-0.271722,0.045196,33
7,0.003846,0.000399,0.001581,0.000619,16.369484,0.39356,"{'enr__alpha': 16.369484273188565, 'enr__l1_ra...",-0.875347,-0.599164,-0.848559,...,-0.795071,-0.778475,-0.644891,-0.823199,-0.517867,-1.014894,-0.796728,-0.785377,0.110622,43
8,0.003596,0.000785,0.001457,0.000505,1.168976,0.604678,"{'enr__alpha': 1.1689757818719282, 'enr__l1_ra...",-0.875347,-0.599164,-0.848559,...,-0.795071,-0.778475,-0.644891,-0.823199,-0.514463,-1.014894,-0.796728,-0.785207,0.111035,42
9,0.003430,0.000879,0.001472,0.000456,57.5297,0.505119,"{'enr__alpha': 57.5297000873286, 'enr__l1_rati...",-0.875347,-0.599164,-0.848559,...,-0.795071,-0.778475,-0.644891,-0.823199,-0.517867,-1.014894,-0.796728,-0.785377,0.110622,43


# Results Visualization

In [9]:
# Print MSE and Settings for optimal model
print('MSE: %.3f' % results_ENLR.best_score_)
print('Config: %s' % results_ENLR.best_params_)

MSE: -0.253
Config: OrderedDict([('enr__alpha', 0.03070124486395089), ('enr__l1_ratio', 1.0)])


In [10]:
results_ENLR.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('enr', ElasticNet(alpha=0.03070124486395089, l1_ratio=1.0))])

In [11]:
Results_LR = pd.DataFrame(list(zip(x.columns.values,results_ENLR.best_estimator_['enr'].coef_)), columns =["Variable", "Coefficient"])
Results_LR

,Variable,Coefficient
0,Employed,0.062873
1,Males,0.000000
2,Vulnerable_pop,0.106105
3,Health_Insurance,0.000000
4,Secondary_Education,-0.000000
5,Life_Expectancy,0.000000
6,prev_cronic_prov,-0.000000
7,Indice_Pobreza_Compuesto,-0.394477
8,White_1000_Inhab,-0.000000
9,Assian_1000_Inhab,0.005713


In [12]:
# Visualization of most important features

Results_LR.loc[Results_LR.Coefficient != 0]

,Variable,Coefficient
0,Employed,0.062873
2,Vulnerable_pop,0.106105
7,Indice_Pobreza_Compuesto,-0.394477
9,Assian_1000_Inhab,0.005713
14,prev_obesidad_prov_endes,0.002109
18,SD_TTtHFH,0.004178
19,Waiting_Time_4Attention_Hours,0.037687
20,SD_WT4AH,0.055146
21,logPD_1000,0.238851
22,Overcrowding,0.022957


In [34]:
def linear_equation(variables, coefficients):
    equation = "y = " + str(np.round(results_ENLR.best_estimator_['enr'].intercept_,2)) + " + "
    for i in range(len(variables)):
        if i > 0:
            if coefficients[i] >= 0:
                equation += " + "
            else:
                equation += " - "
        equation += str(abs(coefficients[i])) + "*" + variables[i]
    return equation

variables = Results_LR.loc[Results_LR.Coefficient != 0].Variable.values
coefficients = np.round(Results_LR.loc[Results_LR.Coefficient != 0].Coefficient.values,2)

print(linear_equation(variables, coefficients))


y = -0.74 + 0.06*Employed + 0.11*Vulnerable_pop - 0.39*Indice_Pobreza_Compuesto + 0.01*Assian_1000_Inhab + 0.0*prev_obesidad_prov_endes + 0.0*SD_TTtHFH + 0.04*Waiting_Time_4Attention_Hours + 0.06*SD_WT4AH + 0.24*logPD_1000 + 0.02*Overcrowding - 0.18*Natural_Region2


# Predict a real case

In [13]:
# Enter data for any province:

x.iloc[0:1,:] # The first province

,Employed,Males,Vulnerable_pop,Health_Insurance,Secondary_Education,Life_Expectancy,prev_cronic_prov,Indice_Pobreza_Compuesto,White_1000_Inhab,Assian_1000_Inhab,...,SD_Days_Till_Attended,Travel_Time_toHFacility_Hours,SD_TTtHFH,Waiting_Time_4Attention_Hours,SD_WT4AH,logPD_1000,Overcrowding,Natural_Region1,Natural_Region2,Natural_Region3
0,421.01755,491.85672,193.42053,837.1167,14840,63.489361,484.55893,47.172291,41.8153,0.09008,...,9.410896,0.409124,0.827978,0.121618,0.492989,0.614774,11.253831,0,1,0


In [14]:
# Print the prediction
n = 98
pred = results_ENLR.best_estimator_.predict(x.iloc[n:(n+1),:]) # Row n point prediction

print("This province would have:", np.round(np.exp(pred),1).item(), "Deaths per 1000 inhabitants")

This province would have: 1.2 Deaths per 1000 inhabitants


In [15]:
# Input data in matrix format
x_test = [[4.2101755e+02, 4.9185672e+02, 1.9342053e+02, 8.3711670e+02,
        1.4840000e+04, 6.3489361e+01, 4.8455893e+02, 4.7172291e+01,
        4.1815300e+01, 9.0080351e-02, 2.4213598e+01, 4.1135788e+02,
        1.0121213e+02, 2.2985615e+01, 1.4377316e+02, 3.3102899e+00,
        9.4108963e+00, 4.0912408e-01, 8.2797784e-01, 1.2161849e-01,
        4.9298859e-01, 6.1477417e-01, 1.1253831e+01, 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00]]

In [16]:
pred = results_ENLR.best_estimator_.predict(x_test) # Row n point prediction

print("This province would have:", np.round(np.exp(pred),1).item(), "Deaths per 1000 inhabitants")

This province would have: 0.6 Deaths per 1000 inhabitants
